# Utilizzo del Item-Based Collaborative Filtering

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd

In [2]:
def cosin_similarity(x, y): 
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        if np.isnan(i) or np.isnan(j):
          continue
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

In [4]:
# lettura dei rating normalizzati dal relativo CSV
pd.set_option('display.max_columns', 500)
normalized_ratings_df = pd.read_csv('./data/normalized_ratings.csv')
# lettura dei rating non normalizzati dal relativo CSV
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')
# se si calcola la similarità tra soli due item corated viene il giusto risultato
# altrimenti i 0 vengono considerati, che facciamo?
#x = normalized_ratings_df.iloc[0, 79:80]
#y = normalized_ratings_df.iloc[1, 79:80]

# la funzione weight_factor prende in input due array quindi ci vanno le [] esterne ad iloc
x = [normalized_ratings_df.iloc[0, 79]]
y = [normalized_ratings_df.iloc[1, 79]]
sim = cosin_similarity(x,y)
print(x)
print(y)
print(sim)
#normalized_ratings_df.head(2)

[-5.988108108]
[6.2337]
-1.0


## Separazione dei rating normalizzati

Si separano i rating noramlizzati in due parti: 
1. una parte contenente tutti gli utenti che hanno valutato tutte e 100 le barzellette;
2. una parte contenente tutti gli utenti che non hanno valutato tutte le barzellette.

Si seleziona poi l'utente target (al quale si vuole fare una raccomandazione) dalla matrice sparsa e si cercano i vicini di quell'utente nella la matrice non sparsa.

In [5]:
# utenti che hanno valutato tutte e 100 le barzellette
complete_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] == 100]
print('totale utenti che hanno valutato tutte le barzellette: ', len(complete_ratings))

# utenti che non hanno valutato tutte le barzellette
sparse_ratings = normalized_ratings_df[normalized_ratings_df['number_of_jokes_rated'] != 100]
print('totale utenti che non hanno valutato tutte le barzellette: ', len(sparse_ratings))

totale utenti che hanno valutato tutte le barzellette:  7200
totale utenti che non hanno valutato tutte le barzellette:  17783


In [6]:
# selezione dell'utente target in maniera random
n = 1000
active_user_id = sparse_ratings.iloc[n, 0]
print("id dell'utente target: {}".format(str(active_user_id)))

id dell'utente target: 1352


In [7]:
print('valutazioni date da utente con id {} per le 100 barzellette'.format(str(active_user_id)))
active_user = sparse_ratings[sparse_ratings['user_id'] == active_user_id]
active_user_rating = active_user.iloc[:, 0:]
active_user_rating

valutazioni date da utente con id 1352 per le 100 barzellette


,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,joke_9,joke_10,joke_11,joke_12,joke_13,joke_14,joke_15,joke_16,joke_17,joke_18,joke_19,joke_20,joke_21,joke_22,joke_23,joke_24,joke_25,joke_26,joke_27,joke_28,joke_29,joke_30,joke_31,joke_32,joke_33,joke_34,joke_35,joke_36,joke_37,joke_38,joke_39,joke_40,joke_41,joke_42,joke_43,joke_44,joke_45,joke_46,joke_47,joke_48,joke_49,joke_50,joke_51,joke_52,joke_53,joke_54,joke_55,joke_56,joke_57,joke_58,joke_59,joke_60,joke_61,joke_62,joke_63,joke_64,joke_65,joke_66,joke_67,joke_68,joke_69,joke_70,joke_71,joke_72,joke_73,joke_74,joke_75,joke_76,joke_77,joke_78,joke_79,joke_80,joke_81,joke_82,joke_83,joke_84,joke_85,joke_86,joke_87,joke_88,joke_89,joke_90,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,100
1351,1352,73,7.871096,-1.888904,-2.568904,-1.118904,2.821096,-4.078904,-0.188904,1.891096,-2.568904,3.931096,5.241096,-1.158904,2.621096,-2.568904,-5.728904,1.071096,1.891096,-5.478904,-0.188904,0.151096,-3.738904,2.621096,-1.458904,4.081096,-3.688904,-4.318904,-5.528904,-3.248904,6.701096,-0.968904,-3.788904,1.991096,-3.248904,-2.858904,4.271096,2.531096,-4.508904,1.701096,4.521096,-4.418904,-4.078904,3.791096,1.361096,2.911096,-2.428904,-4.998904,4.661096,-5.288904,0.971096,-5.438904,3.161096,-2.568904,4.861096,4.421096,3.161096,-0.138904,0.871096,-2.228904,-4.658904,-2.428904,-4.168904,6.551096,-2.908904,-3.448904,8.451096,1.751096,3.931096,4.711096,2.281096,3.641096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.848904,NaN,5.001096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.448904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
active_user_rating_list = active_user_rating.values.ravel()
l = []
# per ogni item che l'utente ha votato si va a vedere 
# la simulitudine con la colonna dell'item da predire
# ad esempio l'utente target non ha votato il joke_93 e quindi andiamo a 
# calcolare le similitudini tra il joke_93 e tutti gli altri joke
for i in range(normalized_ratings_df.shape[1]-3):
    id = "joke_" + str(i+1)
    x = normalized_ratings_df.iloc[:, 93+1].tolist()
    # si prendono tutte le righe (:) della colonna i+2
    y = normalized_ratings_df.iloc[:, i+2].tolist()
    sim = cosin_similarity(x, y)
    l.append(("joke_93",id, sim))
#l[i]è una tripla. l[i][0] è l'id del joke da predire,
#l[i[1]] è l'id del joke confrontato
#l[i][2] è la similitudine
print(l[0])

('joke_93', 'joke_1', -0.024832887060753255)


In [ ]:
# si ordinano i neighbor più simili in ordine decrescente
ind = np.argsort( similarity[:,1] )
similarity = similarity[ind]
similarity.shape

## Si prendono i primi 30 neighbor più simili all'utente target che rispettano una certa soglia (similarità maggiore di 0,1)

In [ ]:
neighbors = similarity[similarity[:,1] > 0.1]
print('Si hanno {} potenziali neighbor! Se ne selezionano 30 tra questi'.format(len(neighbors)))

In [ ]:
# con replace = False si è sicuri che non un neighbor non viene selezionato più volte
index_30_neighbor = np.random.choice(range(len(neighbors)), 30, replace=False)
selected_neighbors = neighbors[index_30_neighbor]
selected_neighbors

## Assegnazione degli score per le barzellette non ancora valutate dall'utente target 

In [ ]:
recommendation_columns = [column for column in active_user_rating.columns if active_user_rating[column].values[0] == 0]

In [ ]:
# si calcola la media delle valutazioni dello user target in modo 
# da avere una predizione più precisa (formula vista nel corso)
active_user_raw_ratings = ratings_df[ratings_df['user_id'] == active_user_id].iloc[:, 2:]
active_user_mean_rating = np.mean(active_user_raw_ratings.drop(recommendation_columns, axis = 1).values)
active_user_mean_rating

In [ ]:
# user id dei neighbor
neighbour_user_id = selected_neighbors[:, 0]

# similarità dei neighbor con lo user target
neighbour_user_similarity = selected_neighbors[:, 1]

print('user id dei neighbor: ', neighbour_user_id, '\n\n')
print('similarità tra utente e neighbor: ', neighbour_user_similarity)

In [ ]:
# si prendono i rating dei neighbor dell'utente target
neighbours_df = complete_ratings[complete_ratings['user_id'].isin(neighbour_user_id)]
len(neighbours_df)

In [ ]:
# si selezionano solo le barzellette che si volgiono raccomandare all'utente (quelle che non ha valutato)
print('Si suggerirà una tra le {} barzellette \n\n'.format(len(recommendation_columns)))

neighbours_df = neighbours_df[recommendation_columns]
neighbours_df.head()

In [ ]:
item_id = recommendation_columns[0]
print('predizione della valutazione della prima barzelletta', item_id)

def score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating ):
    item_rating = neighbours_df[item_id]
    t1, t2 = 0, 0
    for similarity, norm_rating in zip(neighbour_user_similarity, item_rating):
        t1+= norm_rating * similarity
        t2+= similarity
    score = (t1 + active_user_mean_rating)/t2
    return score

# predizione relativa alla prima barzelletta
score_user_item(item_id, neighbours_df,neighbour_user_similarity, active_user_mean_rating )
        

In [ ]:
# raccomandazione della barzelletta con la predizione più alta
top_score = -np.inf
joke_to_suggest = ''

for column in neighbours_df.columns:
    score =score_user_item(column, neighbours_df,neighbour_user_similarity, active_user_mean_rating)
    if score > top_score:
        top_score = score
        joke_to_suggest = column
print('il valore più alto è', top_score)
print('la barzelletta da raccomandare è: ', joke_to_suggest)    